# Finding Flat Plateaus and Sinks in fDOM and turbidity
This will be used to label flat plateaus in the data.

In [11]:
# Imports
import pandas as pd

import sys
sys.path.insert(1, '../')
from Tools import auxiliary_functions
import Tools.data_movement as dm
import Tools.data_processing as dp

In [12]:
# load in data
fDOM_data = dm.read_in_preprocessed_timeseries('../Data/converted_data/julian_format/fDOM_raw_10.1.2011-9.4.2020.csv')
turb_data = dm.read_in_preprocessed_timeseries('../Data/converted_data/julian_format/turbidity_raw_10.1.2011_9.4.2020.csv')
stage_data = dm.read_in_preprocessed_timeseries('../Data/converted_data/julian_format/stage_10.1.11-1.1.19.csv')
stage_data = dp.align_stage_to_fDOM(fDOM_data, stage_data)

## fDOM Plateaus
We need to come up with a function that will find plateaus  
Zach wrote a rudimentary one, needs work

The dataframe this section produces, the index of the peak is the left base (AKA START OF PEAK)


### Get candidates

In [75]:
# Base function attempt
cands = auxiliary_functions.detect_flat_plat(fDOM_data, 100, 40)

indices = []
for i in range(cands.shape[0]):
    if cands[i] == 1:
        indices.append(i)


### Create dataframes

In [73]:
# Create dataframe
# get the beginning and ending of each plateau
last_val = -1
start_idx = -1
end_idx = -1

start_indices = []
end_indices = []

for idx, val in enumerate(indices):
    if val != last_val + 1:
        # we are now in a new peak, save stuff
        start_idx = val
        start_indices.append(start_idx)

        end_idx = last_val
        end_indices.append(end_idx)

    elif idx + 1 == len(indices):
        end_indices.append(val)

    # set last val
    last_val = val

# drop first index in end indices
del end_indices[0]

cands = [[]]
for i in range(len(start_indices)):
    cands.append([start_indices[i], start_indices[i], end_indices[i]])

# create dataframe
cands_df_fdom = pd.DataFrame(cands)
cands_df_fdom.columns = ["idx_of_peak", "left_base", "right_base"]

# drop first row
cands_df_fdom = cands_df_fdom.drop([0])

# set index
cands_df_fdom = cands_df_fdom.set_index("idx_of_peak")

# merge raw data with candidates on peak index
raw_fdom_df = pd.DataFrame(fDOM_data)
raw_fdom_df.columns = ["timestamp_of_peak", "value_of_peak"]
total_df_fdom = cands_df_fdom.join(raw_fdom_df)

# reset index
total_df_fdom = total_df_fdom.reset_index()

# drop unneedeed cols
del total_df_fdom['left_base']
del total_df_fdom['right_base']

total_df_fdom['label_of_peak'] = "NFPT" # set label to be not a flat plateau

total_df_fdom = total_df_fdom.reindex(columns=['timestamp_of_peak', 'value_of_peak', 'label_of_peak', "idx_of_peak"])

print(total_df_fdom)

    timestamp_of_peak  value_of_peak label_of_peak  idx_of_peak
0        2.456144e+06      40.556854          NFPT       9766.0
1        2.456549e+06      52.613096          NFPT      47438.0
2        2.457324e+06      41.105690          NFPT     118793.0
3        2.457359e+06      40.080440          NFPT     122122.0
4        2.457618e+06      63.514980          NFPT     146954.0
5        2.457683e+06      47.797440          NFPT     153248.0
6        2.457690e+06      41.411180          NFPT     153898.0
7        2.458051e+06      53.279340          NFPT     188037.0
8        2.458056e+06      41.736000          NFPT     188525.0
9        2.458060e+06      40.254510          NFPT     188820.0
10       2.458301e+06      47.258600          NFPT     211972.0
11       2.458326e+06      52.316060          NFPT     214407.0
12       2.458328e+06      44.353900          NFPT     214617.0
13       2.458373e+06      40.143810          NFPT     218882.0
14       2.458383e+06      43.556160    

## Turbidity Plateaus

### Get candidate list

In [14]:
# Base function attempt
cands = auxiliary_functions.detect_flat_plat(turb_data, 100, 40)

turb_flat_plat_indxs = []
for i in range(cands.shape[0]):
    if cands[i] == 1:
        turb_flat_plat_indxs.append(i)


33362
67634
103251
195717
200138


### Create dataframes

In [74]:
# create dataframe
last_val = -1
start_idx = -1
end_idx = -1

start_indices = []
end_indices = []

for idx, val in enumerate(turb_flat_plat_indxs):
    if val != last_val + 1:
        # we are now in a new peak, save stuff
        start_idx = val
        start_indices.append(start_idx)

        end_idx = last_val
        end_indices.append(end_idx)

    elif idx + 1 == len(turb_flat_plat_indxs):
        end_indices.append(val)

    # set last val
    last_val = val

# drop first index in end indices
del end_indices[0]

cands = [[]]
for i in range(len(start_indices)):
    cands.append([start_indices[i], start_indices[i], end_indices[i]])

# create dataframe
cands_df_turb = pd.DataFrame(cands)
cands_df_turb.columns = ["idx_of_peak", "left_base", "right_base"]

# drop first row
cands_df_turb = cands_df_turb.drop([0])

# set index
cands_df_turb = cands_df_turb.set_index("idx_of_peak")

# merge raw data with candidates on peak index
raw_turb_df = pd.DataFrame(turb_data)
raw_turb_df.columns = ["timestamp_of_peak", "value_of_peak"]
total_df_turb = cands_df_turb.join(raw_turb_df)

# reset index
total_df_turb = total_df_turb.reset_index()

# drop unneedeed cols
del total_df_turb['left_base']
del total_df_turb['right_base']

total_df_turb['label_of_peak'] = "NFPT" # set label to be not a flat plateau

total_df_turb = total_df_turb.reindex(columns=['timestamp_of_peak', 'value_of_peak', 'label_of_peak', "idx_of_peak"])

print(total_df_turb)

   timestamp_of_peak  value_of_peak label_of_peak  idx_of_peak
0       2.456402e+06      68.036460          NFPT      33361.0
1       2.456759e+06      41.796162          NFPT      67634.0
2       2.457130e+06      93.759808          NFPT     103251.0
3       2.458131e+06      51.682730          NFPT     195717.0
4       2.458177e+06      40.715600          NFPT     200138.0


### Create julian and datetime formats

In [ ]:
# create julian time df

# create datetime df

# convert time


## Output to CSV